<a href="https://colab.research.google.com/github/siy0h/AI-Projects/blob/main/PDF_RAG_Chatbot_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# File Based QA RAG Chatbot
This chatbot leverages LangChain, Streamlit and the OpenAI's ChatGPT API to implement a RAG system with the following features:

- Web based UI
- PDF upload and indexing
- RAG System for analyzing and responding to queriews
- Real time output
- Show document sources from answer of the RAG system

Install App and Dependencies

In [ ]:
!pip install langchain==0.1.12
!pip install langchain-openai==0.0.8
!pip install langchain-community==0.0.29
!pip install streamlit==1.32.2
!pip install PyMuPDF==1.24.0 #to read pdf
!pip install chromadb==0.4.24 #to store embedding
!pip install pyngrok==7.1.5

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import yaml

with open('/openai_credentials.yml', 'r') as file:
    credentials = yaml.safe_load(file)

In [ ]:
credentials.keys()

dict_keys(['openai_key'])

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = credentials['openai_key']

In [ ]:
%%writefile app.py
from langchain_openai import ChatOpenAI, OpenAIEmbeddings #takes chunks and turns them into embeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain_text_splitters import RecursiveCharacterTextSplitter #used to split document into smaller chunks
from langchain_community.vectorstores import Chroma #vector DB
from langchain_core.callbacks import BaseCallbackHandler
import streamlit as st
from operator import itemgetter
import tempfile
import os
import pandas as pd

#initial app landing page
st.set_page_config(page_title="PDF Chatbot", page_icon=":robot:")
st.title("Welcome to File QA RAG Chatbot :robot:")

@st.cache_resource(ttl="1h")

# Takes uploaded PDF documents and extracts the texts
# process and break the text down into smaller chunks
# Passes chunks through an LLM embedder model and create embeddings
# Store document chunks and embeddings into chroma vector database
def configure_retriever(uploaded_files):
    # Read documents
    docs = []
    temp_dir = tempfile.TemporaryDirectory()
    for file in uploaded_files:
      #stores uploaded file in a temporary directory in the server
      temp_file_path = os.path.join(temp_dir.name, file.name)
      with open(temp_file_path, "wb") as f:
        #takes PDF documents and extracts the text
        f.write(uploaded_files[0].getvalue())
    loader = PyMuPDFLoader(temp_file_path)
    docs.extend(loader.load())

    #break down the documents into smaller chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
    chunks = text_splitter.split_documents(docs)

    #create an embedding and store in the Vector DB
    embeddings_model = OpenAIEmbeddings()
    vectorstore = Chroma.from_documents(chunks, embeddings_model)

    #create a retriever object to use later in the RAG process for querying
    retriever = vectorstore.as_retriever()
    return retriever


# Streams the results in real time as it gets responses from ChatGPT
# Shows each token on a UI interface
class StreamHandler(BaseCallbackHandler):
    def __init__(self, container, initial_text: str = ""):
        self.container = container
        self.text = initial_text

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        self.text += token
        self.container.markdown(self.text)



    # creates sidebar to accept PDF uploads

    uploaded_files = st.sidebar.file_uploader(
        label="Upload PDF files", type=["pdf"], accept_multiple_files=True
    )
    if not uploaded_files:
        st.info("Please upload PDF documents to continue.")
        st.stop()

    retriever = configure_retriever(uploaded_files)


    # create a connection ot ChatGPT LLM
    chatgpt = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1,
                         streaming = True)



    # create a prompt template for QA RAG System -> prompt formulated to make sure that
    # the model only answers questions relevant to the retrived documents
    qa_template = """Use the following pieces of information to answer the user's question.
    If you don't know the answer, just say that you don't know. Don't try to make up an answer.
    Use three sentences maximum and keep the answer as concise as possible.

    {context}

    Question: {question}
    Helpful Answer:"""

    qa_prompt = ChatPromptTemplate.from_template(qa_template)

    # formats documents before sending to the LLM. Puts two new lines between every retrieved document
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)


# QA RAG System Chain

# Helps set up input questions and retrieve the relevant contextual docs which
# will be used by ChatGPT to answer the question
        qa_rag_chain = (
        {"context": itemgetter("question") #based on the question, use the retriever to find the closest documents context doc
         |
         retriever
         | format_docs, #puts each retieved document chunks between two lines
         "question": itemgetter("question") #user question
         }
        | qa_prompt #sends the above info to populate the prompt
        | chatgpt #sends the above prompt to chatgpt to generate an answer
    )

        streamlit_msg_history = StreamlitChatMessageHistory(key = "langchain_messages")
        if len(streamlit_msg_history.messages) == 0:
            streamlit_msg_history.add_ai_message("How can I help you?")


        for msg in streamlit_msg_history.messages:
            st.chat_message(msg.type).write(msg.content)

        class PostMessageHandler(BaseCallbackHandler):
            def __init__(self, container, sources):
                self.container = container
                self.sources = []


            def on_retriever_end(self, documents, *, run_id, parent_run_id, **kwargs):

              source_ids = []
              for d in documents:
                metadata = {
                    "source": d.metadata['source'],
                    "page": d.metadata['page'],
                    "content": d.page_content[:200]
                }


                #stores in the empty list only if the source is unique
                idx = (metadata["source"], metadata["page"])
                if idx not in source_ids:
                  source_ids.append(idx)
                  self.sources.append(metadata)

            def on_llm_end(self, response, *, run_id, parent_run_id, **kwargs):
              if len(self.sources)> 0 :
                st.markdown("__Sources__" + "\n")
                st.dataframe(data = pd.DataFrame(self.sources[:3]), width = 1000)

        if user_prompt := st.chat_input():
            st.chat_message("human").write(user_prompt)

            with st.chat_message("ai"):
                stream_handler = StreamHandler(st.empty())
                #tells us where the sources should be displayed

                sources_contrainer = st.write("")
                pm_handler = PostMessageHandler(sources_contrainer)

                config = {"callbacks":[stream_handler,pm_handler]}

                response = qa_rag_chain.invoke({"question":user_prompt},config)







Overwriting app.py


In [ ]:
!streamlit run app.py --server.port=8989 &>./logs.txt &

In [ ]:
!pip install pyngrok
from pyngrok import ngrok

ngrok.kill()

with open('/content/ngrok_credentials.yml', 'r') as file:
    ngrok_credentials = yaml.safe_load(file)

ngrok_credentials.keys()
ngrok_auth_token = ngrok_credentials['authtoken']

ngrok.set_auth_token(ngrok_auth_token)
ngrok_tunnel = ngrok.connect(8989)

print("Streamlit App:", ngrok_tunnel.public_url)



Streamlit App: https://64dc-35-185-158-175.ngrok-free.app
